# Data Generation (Dex Mimic Gen)

Task 1:
1. Coordinated Task - Pickup Tray

- [x] Load a dual-robot environment on Robosuite
- [x] Record a dual-robot random-action video
- [x] Extract a demonstration from the given dataset and visualize
- [x] Augment more object states and Visualize original trajectory execution on them
- [ ] Implement DexMimicGen's data aug strategy and visualize the new trajectories on these augmented states|

In [1]:
import robosuite 
import dexmimicgen  

import h5py 
import imageio 
import numpy as np
import time 
import os
import json

[robosuite WARNING] No private macro file found! (macros.py:57)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
[robosuite WARNING] To setup, run: python /home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/scripts/setup_macros.py (macros.py:59)
[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


In [2]:
'''

1. Loading dual robot environment
'''

def get_env_metadata_from_dataset(dataset_path, ds_format="robomimic"):
    """
    Retrieves env metadata from dataset.

    Args:
        dataset_path (str): path to dataset

    Returns:
        env_meta (dict): environment metadata. Contains 3 keys:

            :`'env_name'`: name of environment
            :`'type'`: type of environment, should be a value in EB.EnvType
            :`'env_kwargs'`: dictionary of keyword arguments to pass to environment constructor
    """
    dataset_path = os.path.expanduser(dataset_path)
    f = h5py.File(dataset_path, "r")
    if ds_format == "robomimic":
        env_meta = json.loads(f["data"].attrs["env_args"])
    else:
        raise ValueError
    f.close()
    return env_meta

dataset_path = "/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/dexmimicgen/datasets/two_arm_box_cleanup.hdf5"

env_meta = get_env_metadata_from_dataset(dataset_path)
print(env_meta)

{'env_name': 'TwoArmBoxCleanup', 'env_version': '1.5.1', 'type': 1, 'env_kwargs': {'robots': ['PandaDexRH', 'PandaDexLH'], 'controller_configs': {'type': 'BASIC', 'body_parts': {'right': {'type': 'OSC_POSE', 'input_max': 1, 'input_min': -1, 'output_max': [0.05, 0.05, 0.05, 0.5, 0.5, 0.5], 'output_min': [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5], 'kp': 150, 'damping_ratio': 1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_ratio_limits': [0, 10], 'position_limits': None, 'orientation_limits': None, 'uncouple_pos_ori': True, 'input_type': 'delta', 'input_ref_frame': 'base', 'interpolation': None, 'ramp_ratio': 0.2, 'gripper': {'type': 'GRIP', 'use_action_scaling': False}}}}, 'translucent_robot': False, 'env_configuration': 'parallel', 'reward_shaping': False, 'camera_names': ['agentview', 'robot0_eye_in_hand', 'robot1_eye_in_hand'], 'camera_heights': 84, 'camera_widths': 84, 'has_renderer': False, 'has_offscreen_renderer': True, 'ignore_done': True, 'use_object_obs': True, 'use_

In [3]:
write_to_video = False

env_kwargs = env_meta['env_kwargs']
env_kwargs["env_name"] = env_meta["env_name"]
env_kwargs["has_renderer"] = True
env_kwargs["renderer"] = "mjviewer"
env_kwargs["has_offscreen_renderer"] = write_to_video # False # write_video
env_kwargs["use_camera_obs"] = False

env_kwargs.pop("env_lang")

env = robosuite.make(**env_kwargs)

In [4]:
# / = env.reset()

In [4]:
env.action_dim
env.reset() # reset kills the viewer. So, do not just reset stuff. Instead, reset, then set action to zero, and then render
# env.render()
env.step(np.zeros(env.action_dim))
env.render()

In [90]:
print(write_to_video)

False


In [91]:
'''
Run without writing a video
'''

if not write_to_video:
    for i in range(1000):
        action = np.random.randn(env.action_dim) # 24
        obs, reward, done, info = env.step(action)
        env.render()
else:
    video_filename = "two_arm_box_cleanup_random.mp4"
    video_path = os.path.join("/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/dev/saved_videos", video_filename)
    print(f"Saving video to {video_path}")
    writer = imageio.get_writer(video_path, fps=30)
    for i in range(100):
        action = np.random.randn(env.action_dim) # 24
        obs, reward, done, info = env.step(action)
        frame = env.sim.render(height=512, width=512, camera_name=None)
        # Frame generated is inverted (just upside down)
        frame = np.flipud(frame)
        # frame = np.fliplr(frame)
        writer.append_data(frame)
    writer.close()

In [5]:
'''
Extracting a demonstration from the dataset
'''

data = h5py.File(dataset_path, "r")
print(f"Keys in the dataset: {data['data'].keys()}")
print(f"Number of demonstrations in the dataset: {len(data['data'].keys())}")
demo_id = 'demo_10'
print(f"Length of demo {demo_id}: {data['data'][demo_id]['obs']['robot0_eef_pos'].shape[0]}")
print(f"Observations shape: {data['data'][demo_id]['obs'].keys()}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Actions: {data['data'][demo_id]['actions']}")
print(f"Actions shape: {data['data'][demo_id]['actions'].shape}")
print(f"Action at step 0: {data['data'][demo_id]['actions'][0]}")

action_trajectory = data['data'][demo_id]['actions']
obs_trajectory = data['data'][demo_id]['obs']
states_trajectory = data['data'][demo_id]['states']

Keys in the dataset: <KeysViewHDF5 ['demo_0', 'demo_1', 'demo_10', 'demo_100', 'demo_1000', 'demo_1001', 'demo_1002', 'demo_1003', 'demo_1004', 'demo_1005', 'demo_1006', 'demo_1007', 'demo_1008', 'demo_1009', 'demo_101', 'demo_1010', 'demo_1011', 'demo_1012', 'demo_1013', 'demo_1014', 'demo_1015', 'demo_102', 'demo_103', 'demo_104', 'demo_105', 'demo_106', 'demo_107', 'demo_108', 'demo_109', 'demo_11', 'demo_110', 'demo_111', 'demo_112', 'demo_113', 'demo_114', 'demo_115', 'demo_116', 'demo_117', 'demo_118', 'demo_119', 'demo_12', 'demo_120', 'demo_121', 'demo_122', 'demo_123', 'demo_124', 'demo_125', 'demo_126', 'demo_127', 'demo_128', 'demo_129', 'demo_13', 'demo_130', 'demo_131', 'demo_132', 'demo_133', 'demo_134', 'demo_135', 'demo_136', 'demo_137', 'demo_138', 'demo_139', 'demo_14', 'demo_140', 'demo_141', 'demo_142', 'demo_143', 'demo_144', 'demo_145', 'demo_146', 'demo_147', 'demo_148', 'demo_149', 'demo_15', 'demo_150', 'demo_151', 'demo_152', 'demo_153', 'demo_154', 'demo_155'

In [6]:
# Reset simulation to the starting state of the demonstration
env.reset()
env.sim.set_state_from_flattened(states_trajectory[25])
env.sim.forward()

In [7]:
env.render()
env.step(np.zeros(env.action_dim))
env.render()

In [8]:
def just_reset(env, state):
    # The following resets the state
    env.reset()
    # env.sim.set_state_from_flattened(state)
    # env.sim.forward()

    # This part is just to visualize the state after resetting
    env.render()
    env.step(np.zeros(env.action_dim))
    env.render()

def reset_to_state(env, state):
    # The following resets the state
    env.reset()
    env.sim.set_state_from_flattened(state)
    env.sim.forward()

    # This part is just to visualize the state after resetting
    env.render()
    env.step(np.zeros(env.action_dim))
    env.render()

In [11]:
'''
Replay a demonstration
'''
# reset_to_state(env, states_trajectory[0])
just_reset(env, states_trajectory[25])

for i in range(action_trajectory.shape[0]):
    action = action_trajectory[i]
    obs, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.05)

In [105]:
'''

Perturb object states and visualize original trajectory execution on them
'''

# Reset to initial state
reset_to_state(env, states_trajectory[0])
# Perturb object states

env.reset() # This alone perturbs the object states
new_initial_state = env.get_state()

# initial_box_pos = env.sim.data.get_body_xpos('box')
# initial_lid_pos = env.sim.data.get_body_xpos('lid')
# print(f"Initial box pos: {initial_box_pos}, Initial lid pos: {initial_lid_pos}")
# env.sim.data.set_joint_qpos('box_x', initial_box_pos[0] + 0.1)
# env.sim.data.set_joint_qpos('box_y', initial_box_pos[1] + 0.1)
# env.sim.data.set_joint_qpos('lid_x', initial_lid_pos[0] + 0.1)
# env.sim.data.set_joint_qpos('lid_y', initial_lid_pos[1] + 0.1)
# env.sim.forward()
# perturbed_box_pos = env.sim.data.get_body_xpos('box')
# perturbed_lid_pos = env.sim.data.get_body_xpos('lid')
# print(f"Perturbed box pos: {perturbed_box_pos}, Perturbed lid pos: {perturbed_lid_pos}")        

# env.step(np.zeros(env.action_dim))
# env.render()

In [10]:
print(data['data']['demo_10']['datagen_info']['object_poses'].keys())

src_box_pose = data['data']['demo_10']['datagen_info']['object_poses']['box'][0]
src_lid_pose = data['data']['demo_10']['datagen_info']['object_poses']['lid'][0]

print(f"Source box pose: {src_box_pose}")
print(f"Source lid pose: {src_lid_pose}, type: {type(src_lid_pose)}")

<KeysViewHDF5 ['box', 'lid']>
Source box pose: [[ 1.          0.          0.         -0.04543385]
 [ 0.          1.          0.         -0.15      ]
 [ 0.          0.          1.          0.776     ]
 [ 0.          0.          0.          1.        ]]
Source lid pose: [[ 1.          0.          0.         -0.00157158]
 [ 0.          1.          0.          0.2       ]
 [ 0.          0.          1.          0.725     ]
 [ 0.          0.          0.          1.        ]], type: <class 'numpy.ndarray'>


In [11]:
src_eef_poses = data['data']['demo_10']['datagen_info']['eef_pose']

In [12]:
data['data']['demo_10']['datagen_info']['gripper_action'].shape

src_gripper_actions0 = data['data']['demo_10']['datagen_info']['gripper_action'][:, 0, :]
src_gripper_actions1 = data['data']['demo_10']['datagen_info']['gripper_action'][:, 1, :]

In [27]:
env.reset()

OrderedDict([('robot0_joint_pos',
              array([ 0.01276286,  0.18224139, -0.01374993, -2.61794417,  0.01239359,
                      2.93534159,  0.77909893])),
             ('robot0_joint_pos_cos',
              array([ 0.99991856,  0.98343995,  0.99990547, -0.86600055,  0.9999232 ,
                     -0.97880554,  0.71154695])),
             ('robot0_joint_pos_sin',
              array([ 0.01276251,  0.1812343 , -0.0137495 , -0.50004305,  0.01239327,
                      0.20479187,  0.70263855])),
             ('robot0_joint_vel', array([0., 0., 0., 0., 0., 0., 0.])),
             ('robot0_joint_acc',
              array([ 3.67855719e-03,  4.73465663e+00,  4.46156545e-02, -2.65234125e+00,
                      8.44609256e-04,  5.85293249e-03,  1.34220805e-03])),
             ('robot0_eef_pos',
              array([-0.113745  , -0.25146517,  1.11293711])),
             ('robot0_eef_quat',
              array([ 9.97712265e-01, -2.87240972e-03,  6.75422325e-02, -1.77688667e

In [28]:
'''
'box_obj_root', 'lid_obj_root', 'lid_obj_box_1_main', 'lid_obj_box_2_main'
'''

obj_name = 'box_obj_root'

'''This the other main object that is also being tracked
'''

obj_id = env.sim.model.body_name2id(obj_name)
obj_pos = np.array(env.sim.data.body_xpos[obj_id])
obj_rot = np.array(env.sim.data.body_xmat[obj_id].reshape(3, 3))
print(f"Object name: {obj_name}, Object id: {obj_id}, Object pos: {obj_pos}, Object rot: {obj_rot}")

new_obj_pose = {
    'pos': obj_pos,
    'rot': obj_rot
}
np.save('box_object_pose_test.npy', new_obj_pose, allow_pickle=True)
box_obj_pose_loaded = np.load('box_object_pose_test.npy', allow_pickle=True).item()
print(box_obj_pose_loaded)

Object name: box_obj_root, Object id: 74, Object pos: [ 0.03555458 -0.15        0.776     ], Object rot: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
{'pos': array([ 0.03555458, -0.15      ,  0.776     ]), 'rot': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])}


In [29]:
obj_name = 'lid_obj_root'

'''This is the main object that is being tracked'''

obj_id = env.sim.model.body_name2id(obj_name)
obj_pos = np.array(env.sim.data.body_xpos[obj_id])
obj_rot = np.array(env.sim.data.body_xmat[obj_id].reshape(3, 3))
print(f"Object name: {obj_name}, Object id: {obj_id}, Object pos: {obj_pos}, Object rot: {obj_rot}")

new_obj_pose = {
    'pos': obj_pos,
    'rot': obj_rot
}
np.save('lid_object_pose_test.npy', new_obj_pose, allow_pickle=True)
lid_obj_pose_loaded = np.load('lid_object_pose_test.npy', allow_pickle=True).item()
print(lid_obj_pose_loaded)

Object name: lid_obj_root, Object id: 75, Object pos: [0.04160595 0.2        0.725     ], Object rot: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
{'pos': array([0.04160595, 0.2       , 0.725     ]), 'rot': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])}


In [30]:
obj_name = 'lid_obj_box_1_main'

obj_id = env.sim.model.body_name2id(obj_name)
obj_pos = np.array(env.sim.data.body_xpos[obj_id])
obj_rot = np.array(env.sim.data.body_xmat[obj_id].reshape(3, 3))
print(f"Object name: {obj_name}, Object id: {obj_id}, Object pos: {obj_pos}, Object rot: {obj_rot}")

Object name: lid_obj_box_1_main, Object id: 76, Object pos: [0.04160595 0.2        0.72      ], Object rot: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [31]:
obj_name = 'lid_obj_box_2_main'

obj_id = env.sim.model.body_name2id(obj_name)
obj_pos = np.array(env.sim.data.body_xpos[obj_id])
obj_rot = np.array(env.sim.data.body_xmat[obj_id].reshape(3, 3))
print(f"Object name: {obj_name}, Object id: {obj_id}, Object pos: {obj_pos}, Object rot: {obj_rot}")

Object name: lid_obj_box_2_main, Object id: 77, Object pos: [0.04160595 0.2        0.77      ], Object rot: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [32]:
new_obj_pose = {
    'pos': obj_pos,
    'rot': obj_rot
}

np.save('new_object_pose_test.npy', new_obj_pose, allow_pickle=True)
new_obj_pose_loaded = np.load('new_object_pose_test.npy', allow_pickle=True).item()
print(new_obj_pose_loaded)

{'pos': array([0.04160595, 0.2       , 0.77      ]), 'rot': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])}


In [33]:
from pose_utils_vishal_from_mimicgen import transform_source_data_segment_using_object_pose, make_pose


lid_pose_new = make_pose(lid_obj_pose_loaded['pos'], lid_obj_pose_loaded['rot'])
# cur_lid_pose = make_pose()

box_pose_new = make_pose(box_obj_pose_loaded['pos'], box_obj_pose_loaded['rot'])
# cur_box_pose = make_pose()

In [34]:
target_traj_arm0 = transform_source_data_segment_using_object_pose (
    obj_pose=lid_pose_new,
    src_eef_poses=src_eef_poses[:, 0, :, :],
    src_obj_pose=src_lid_pose
)

target_traj_arm1 = transform_source_data_segment_using_object_pose (
    obj_pose=box_pose_new,
    src_eef_poses=src_eef_poses[:, 1, :, :],
    src_obj_pose=src_box_pose
)

In [35]:
from waypoint_utils_vishal_from_mimicgen import Waypoint, WaypointSequence, WaypointTrajectory

In [36]:
src_gripper_actions0.shape
src_gripper_actions0[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       1.57079633])

In [37]:
waypoint_obj0 = WaypointTrajectory()
waypoint_obj1 = WaypointTrajectory()

# waypoint_obj0.add_waypoint_sequence()
# waypoint_obj1.add_waypoint_sequence()

'''
Manually interpolating waypoints
'''
num_steps = 20
initial_poses_0 = np.array([target_traj_arm0[0, :, :] for _ in range(num_steps)])
initial_gripper_actions_0 = np.array([[src_gripper_actions0[0]] for _ in range(num_steps)])

initial_poses_1 = np.array([target_traj_arm1[0, :, :] for _ in range(num_steps)])
initial_gripper_actions_1 = np.array([[src_gripper_actions1[0]] for _ in range(num_steps)])

# waypoint_obj0.add_waypoint_sequence_for_target_pose(pose=src_eef_poses[0, 0, :, :], gripper_action=src_gripper_actions0[0], num_steps=20, skip_interpolation=True)
# waypoint_obj1.add_waypoint_sequence_for_target_pose(pose=src_eef_poses[0, 1, :, :], gripper_action=src_gripper_actions1[0], num_steps=20, skip_interpolation=True)

In [45]:
# reset_to_state(env, new_initial_state)
import robosuite.utils.transform_utils as T
import pose_utils_vishal_from_mimicgen as PoseUtils
def target_pose_to_action(env, target_pose, relative=True):
    """
    Takes a target pose for the end effector controller and returns an action 
    (usually a normalized delta pose action) to try and achieve that target pose. 

    Args:
        target_pose (np.array): 4x4 target eef pose
        relative (bool): if True, use relative pose actions, else absolute pose actions

    Returns:
        action (np.array): action compatible with env.step (minus gripper actuation)
    """

    # version check for robosuite - must be v1.2+, so that we're using the correct controller convention
    assert (robosuite.__version__.split(".")[0] == "1")
    assert (robosuite.__version__.split(".")[1] >= "2")

    # target position and rotation
    target_pos, target_rot = PoseUtils.unmake_pose(target_pose)

    # current position and rotation
    curr_pose = env.get_robot_eef_pose()
    curr_pos, curr_rot = PoseUtils.unmake_pose(curr_pose)

    # get maximum position and rotation action bounds
    max_dpos = env.robots[0].controller.output_max[0]
    max_drot = env.robots[0].controller.output_max[3]

    if relative:
        # normalized delta position action
        delta_position = target_pos - curr_pos
        delta_position = np.clip(delta_position / max_dpos, -1., 1.)

        # normalized delta rotation action
        delta_rot_mat = target_rot.dot(curr_rot.T)
        delta_quat = T.mat2quat(delta_rot_mat)
        delta_rotation = T.quat2axisangle(delta_quat)
        delta_rotation = np.clip(delta_rotation / max_drot, -1., 1.)
        return np.concatenate([delta_position, delta_rotation])

    # absolute position and rotation action
    target_quat = T.mat2quat(target_rot)
    abs_rotation = T.quat2axisangle(target_quat)
    return np.concatenate([target_pos, abs_rotation])

# Execute trajectory
target_pose_to_action(env, target_traj_arm0[0, :, :])


AttributeError: 'TwoArmBoxCleanup' object has no attribute 'get_robot_eef_pose'

In [ ]:
env.robots[0].

{'right': <robosuite.utils.buffers.DeltaBuffer at 0x7a773f623730>}

In [42]:
data['data']['demo_10']['actions']

<HDF5 dataset "actions": shape (226, 24), type "<f8">

In [ ]:
# env.robots

In [50]:
env.get_state()['states']

array([ 0.        ,  0.02154552,  0.20462182, -0.00419658, -2.63458657,
        0.02531663,  2.90608111,  0.78614903,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.0086681 ,  0.20627634,  0.01441064, -2.6088026 ,  0.01159211,
        2.90212497,  0.77122992,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.02114113,
       -0.15      ,  0.776     ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.01714837,  0.2       ,  0.725     ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [36]:
env.reset()
new_initial_state1 = env.get_state()
print(new_initial_state1['states'])
env.reset()
new_initial_state2 = env.get_state()
print(new_initial_state2['states'])

[ 0.00000000e+00  4.94176412e-02  1.89245364e-01 -2.42125606e-02
 -2.63561166e+00  1.75507958e-02  2.95030488e+00  7.94999070e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -8.78015720e-03  1.82793945e-01 -1.33794683e-03 -2.60353444e+00
 -1.89915082e-02  2.92161712e+00  7.99628537e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  2.72084346e-02
 -1.50000000e-01  7.76000000e-01  1.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -4.94923513e-02  2.00000000e-01
  7.25000000e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

In [42]:
new_initial_state1['model']

'<mujoco model="base">\n  <compiler angle="radian" meshdir="meshes/"/>\n\n  <option impratio="20" density="1.2" viscosity="2e-05" cone="elliptic"/>\n\n  <size njmax="5000" nconmax="5000"/>\n\n  <visual>\n    <map znear="0.001"/>\n  </visual>\n\n  <asset>\n    <texture type="skybox" colorspace="auto" builtin="gradient" rgb1="0.9 0.9 1" rgb2="0.2 0.3 0.4" width="256" height="1536"/>\n    <texture type="2d" colorspace="sRGB" name="texplane" file="/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/models/assets/textures/light-gray-floor-tile.png"/>\n    <texture type="cube" colorspace="linear" name="tex-ceramic" file="/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/models/assets/textures/ceramic.png"/>\n    <texture type="cube" colorspace="linear" name="tex-steel-brushed" file="/home/vishal/Volume_E/Active/Gap_year/grand-project-2025/deepak/robosuite/robosuite/models/assets/textures/steel-brushed.png"/>\n    <texture type

In [38]:
np.abs(new_initial_state1['states'] - new_initial_state2['states'])

array([0.        , 0.02787212, 0.01537646, 0.02001598, 0.00102509,
       0.00776584, 0.04422376, 0.00885004, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01744826, 0.02348239, 0.01574859, 0.00526816, 0.03058362,
       0.01949215, 0.02839862, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00606731,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.06664072, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [57]:
env.reset()
new_initial_state3 = env.get_state()

obj_name = 'box_obj_root'

'''This is the main object that is being tracked'''

obj_id = env.sim.model.body_name2id(obj_name)
obj_pos = np.array(env.sim.data.body_xpos[obj_id])
obj_rot = np.array(env.sim.data.body_xmat[obj_id].reshape(3, 3))
print(f"Object name: {obj_name}, Object id: {obj_id}, Object pos: {obj_pos}, Object rot: {obj_rot}")

obj_name = 'lid_obj_root'

obj_id = env.sim.model.body_name2id(obj_name)
obj_pos = np.array(env.sim.data.body_xpos[obj_id])
obj_rot = np.array(env.sim.data.body_xmat[obj_id].reshape(3, 3))
print(f"Object name: {obj_name}, Object id: {obj_id}, Object pos: {obj_pos}, Object rot: {obj_rot}")

env.render()
env.step(np.zeros(env.action_dim))
env.render()


Object name: box_obj_root, Object id: 74, Object pos: [ 0.01547843 -0.15        0.776     ], Object rot: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Object name: lid_obj_root, Object id: 75, Object pos: [-0.03751437  0.2         0.725     ], Object rot: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
'''
So, from what I understand from MimicGen, new object state is sampled at every reset. 

Just use the pose created by reset for datageneration.
'''